In [1]:
# SVM Classification on MNIST Dataset

# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import utils  # This should contain necessary utility functions for loading data and saving results
import os

In [2]:
# Paths to the dataset CSV files
train_path = '../data/train/mnist_train_small.csv'
test_path = '../data/test/mnist_test.csv'

# Load the data using the utility functions
X_train, y_train, X_test, y_test = utils.load_data(train_path, test_path)

# Check the shape of the datasets
print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

Training data shape: (19999, 784)
Test data shape: (9999, 784)


In [3]:

# Initialize and train the SVM classifier
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')  # You can try other kernels like 'linear', 'poly', etc.
svm_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test)


In [4]:
# Generate confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print classification report to the console
print("Classification Report:\n", class_report)


Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.98       980
           1       0.98      0.99      0.99      1135
           2       0.97      0.97      0.97      1032
           3       0.96      0.97      0.97      1010
           4       0.96      0.97      0.97       982
           5       0.97      0.97      0.97       892
           6       0.98      0.98      0.98       958
           7       0.97      0.96      0.96      1027
           8       0.97      0.95      0.96       974
           9       0.96      0.95      0.96      1009

    accuracy                           0.97      9999
   macro avg       0.97      0.97      0.97      9999
weighted avg       0.97      0.97      0.97      9999



In [5]:
# Save classification report and confusion matrix in the results folder
utils.save_classification_report(class_report, 'results/classification_report.txt')

# Save confusion matrix as a heatmap image
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title('Confusion Matrix - SVM')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('results/confusion_matrix.png')
plt.show()


AttributeError: module 'utils' has no attribute 'save_classification_report'

In [ ]:
# Optional: Save predictions to CSV for further analysis
test_predictions_df = pd.DataFrame({'True Label': y_test, 'Predicted Label': y_pred})
test_predictions_df.to_csv('results/test_predictions.csv', index=False)

# Optional: Validation accuracy
accuracy = np.mean(y_test == y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

# Save accuracy to a text file
with open('results/validation_accuracy.txt', 'w') as f:
    f.write(f"Validation Accuracy: {accuracy:.4f}")

In [ ]:
!pip install shap

In [ ]:
import shap 

In [ ]:
# Create SHAP explainer
explainer = shap.KernelExplainer(svm_model.predict, X_train[:100])  # Limit sample size for efficiency

# Calculate SHAP values for the test data
shap_values = explainer.shap_values(X_test[:100])


In [ ]:
# SHAP dependence plot for a specific feature
shap.dependence_plot(0, shap_values, X_test[:100], feature_names=feature_names)  # Feature index 0 (change as needed)


In [ ]:
import matplotlib.pyplot as plt

# Save SHAP summary plot
shap.summary_plot(shap_values, X_test[:100], feature_names=feature_names, show=False)
plt.savefig('./results/shap_summary_plot.png')

# Save SHAP dependence plot for a specific feature
shap.dependence_plot(0, shap_values, X_test[:100], feature_names=feature_names, show=False)
plt.savefig('./results/shap_dependence_plot_feature_0.png')


In [ ]:
# SHAP force plot for a single prediction
shap.force_plot(explainer.expected_value, shap_values[0], X_test[0], feature_names=feature_names)
